In [2]:
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv('../data/data_predictiions.csv')
df['date_received'] = pd.to_datetime(df['date_received'])

In [4]:
region_mapping = {
    'Northeast': ['CT', 'ME', 'MA', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    'Southeast': ['AL', 'AR', 'DE', 'FL', 'GA', 'KY', 'LA', 'MD', 'MS', 'NC', 'SC', 'TN', 'VA', 'WV'],
    'Midwest': ['IL', 'IN', 'IA', 'KS', 'MI', 'MN', 'MO', 'NE', 'ND', 'OH', 'SD', 'WI'],
    'Southwest': ['AZ', 'NM', 'OK', 'TX'],
    'West': ['AK', 'CA', 'CO', 'HI', 'ID', 'MT', 'NV', 'OR', 'UT', 'WA', 'WY']
}
state_to_region = {state: region for region, states in region_mapping.items() for state in states}
df['region'] = df['state'].map(state_to_region)

In [5]:
state_data = df.groupby(['state', 'predicted_category']).size().reset_index(name='count')

fig = px.choropleth(
    state_data,
    locations='state',
    locationmode='USA-states',
    color='count',
    animation_frame='predicted_category',
    scope='usa',
    title='Complaints by State and Category'
)
fig.show()